In [ ]:
import ee
import geemap.core as geemap
import numpy as np


# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='geostat-test')

In [ ]:
import geemap.core as geemap

In [ ]:
# Band aliases.
BLUE = 'CMI_C01'
RED = 'CMI_C02'
VEGGIE = 'CMI_C03'
CIRRUS = 'CMI_C04'
SNOW = 'CMI_C05'
PARTICLE = 'CMI_C06'
SHORTWAVE = 'CMI_C07'
UPPER = 'CMI_C08'
MID = 'CMI_C09'
LOWER = 'CMI_C10'
CLOUDTOP = 'CMI_C11'
OZONE = 'CMI_C12'
LONGWAVE = 'CMI_C13'
SEATEMP = 'CMI_C14'
ASH = 'CMI_C15'
CO2 = 'CMI_C16'
GREEN = 'GREEN'

# 16 pairs of CMI and DQF followed by Bah 2018 synthetic green.
# Band numbers in the EE asset, 0-based.
NUM_BANDS = 33

# Skipping the interleaved DQF bands.
BLUE_BAND_INDEX = (1 - 1) * 2
RED_BAND_INDEX = (2 - 1) * 2
VEGGIE_BAND_INDEX = (3 - 1) * 2
GREEN_BAND_INDEX = NUM_BANDS - 1

# Visualization range for GOES RGB.
GOES_MIN = 0.0
GOES_MAX = 0.7  # Alternatively 1.0 or 1.3.
GAMMA = 1.3

goesRgbViz = {
    'bands': [CIRRUS, CO2, CLOUDS],
    'min': [0,92,127],
    'max': [1.3,319,342],
    'gamma': [1,0.5,2]
}

def apply_scale_and_offset(image):
    image = ee.Image(image)
    bands = [None] * NUM_BANDS
    for i in range(1, 17):
        band_name = 'CMI_C' + str(100 + i)[-2:]
        offset = ee.Number(image.get(band_name + '_offset'))
        scale = ee.Number(image.get(band_name + '_scale'))
        bands[(i - 1) * 2] = image.select(band_name).multiply(scale).add(offset)

        dqf_name = 'DQF_C' + str(100 + i)[-2:]
        bands[(i - 1) * 2 + 1] = image.select(dqf_name)

    # Bah, Gunshor, Schmit, Generation of GOES-16 True Color Imagery without a
    # Green Band, 2018. https://doi.org/10.1029/2018EA000379
    # Green = 0.45 * Red + 0.10 * NIR + 0.45 * Blue
    green1 = bands[RED_BAND_INDEX].multiply(0.45)
    green2 = bands[VEGGIE_BAND_INDEX].multiply(0.10)
    green3 = bands[BLUE_BAND_INDEX].multiply(0.45)
    green = green1.add(green2).add(green3)
    bands[GREEN_BAND_INDEX] = green.rename(GREEN)

    return ee.Image(ee.Image(bands).copyProperties(image, image.propertyNames()))

In [ ]:
goesCollection = ee.ImageCollection('NOAA/GOES/16/MCMIPF')
ids = goesCollection.aggregate_array('system:index').getInfo()

In [ ]:
collection = 'NOAA/GOES/16/MCMIPF/'
asset_id = collection + ids[100100]
image_vis = apply_scale_and_offset(asset_id)
image = ee.Image(asset_id)

In [ ]:
# Create an NDWI image, define visualization parameters and display.
# ndwi = image.normalizedDifference(['CMI_C10', 'CMI_C02'])
ndwi = image.normalizedDifference([BLUE, RED])
ndwi_viz = {'min': 0.5, 'max': 1, 'palette': ['00FFFF', '0000FF']}

# Define a map centered on San Francisco Bay.
map_ndwi = geemap.Map(center=[37.5010, -122.1899], zoom=10)

# Add the image layer to the map and display it.
map_ndwi.add_layer(ndwi, ndwi_viz, 'NDWI')

In [ ]:
# Mask the non-watery parts of the image, where NDWI < 0.4.
ndwi_masked = ndwi.updateMask(ndwi.gte(0.4))

# Define a map centered on San Francisco Bay.
map_ndwi_masked = geemap.Map(center=[22.5010, -85.1899], zoom=5)

# Add the image layer to the map and display it.
map_ndwi_masked.add_layer(ndwi_masked, ndwi_viz, 'NDWI masked')
display(map_ndwi_masked)